In [1]:
import nest_asyncio

nest_asyncio.apply()
import time, random
from pydantic import BaseModel

In [22]:
import asyncio

import uuid

ModuleNotFoundError: No module named 'QueueEmpty'

In [3]:
random.random()

0.11495891351016496

In [4]:
time.sleep(0.1)

In [5]:
class LL(BaseModel):
    id: str = uuid.uuid4().hex
    worker_id: int
    letter: str

In [6]:
a = LL(id=uuid.uuid4().hex, worker_id=1, letter="a")

In [7]:
type(a.model_dump())

dict

In [8]:
async def worker(word, queue, woker_id):
    for i in word:
        t = random.random()
        print('worker: ', i, " sleeping for ", t)
        await asyncio.sleep(random.random())
        await queue.put(
            LL(id=uuid.uuid4().hex, worker_id=woker_id, letter=i)
        )
    return

In [9]:
async def mainer():
    queue = asyncio.Queue()
    word = ["hello", "world", "red", "blue", "green"]
    tasks = [worker(word[i], queue, i) for i in range(2)]
    await asyncio.gather(*tasks)
    # while not queue.empty():
    #     print(queue.get())
    return queue

In [10]:
out = await mainer()

worker:  h  sleeping for  0.5739210534851691
worker:  w  sleeping for  0.44072361666164817
worker:  e  sleeping for  0.5045108104852616
worker:  o  sleeping for  0.6400272756464778
worker:  l  sleeping for  0.03617773175313266
worker:  r  sleeping for  0.9491729520935999
worker:  l  sleeping for  0.6352896471127185
worker:  l  sleeping for  0.41614391993496935
worker:  d  sleeping for  0.16073162858860057
worker:  o  sleeping for  0.12703046685091313


In [21]:
out.empty()
print(out.get_nowait())

QueueEmpty: 

In [16]:
# 'empty', 'full', 'get', 'get_nowait', 'join', 'maxsize', 'put', 'put_nowait', 'qsize', 'task_done'

In [14]:
out.maxsize

0

In [57]:
import asyncio

async def _producer(queue, n):
    for i in range(n):
        await asyncio.sleep(1)  # Simulate some work with a sleep
        item = f'item-{i}'
        await queue.put(item)
        print(f'Produced {item}')
        

async def producer(queue, n):
    asyncio.create_task(_producer(queue, n))
        
def consumer(queue):
    asyncio.run(_consumer(queue))

async def _consumer(queue):
    while True:
        item = await queue.get()
        await asyncio.sleep(1)
        if item is None:  # We use `None` to signal the consumer to stop
            break
        print(f'Consumed {item}')
        queue.task_done()

async def main():
    queue = asyncio.Queue()
    n = 10  # Number of items to produce

    producer_task = asyncio.create_task(producer(queue, n))
    consumer_task = asyncio.create_task(consumer(queue))

    await producer_task
    await queue.put(None)  # Signal the consumer to stop
    await consumer_task

# Run the main function
asyncio.run(main())


Produced item-0
Produced item-1
Consumed item-0
Produced item-2
Consumed item-1
Produced item-3
Consumed item-2
Produced item-4
Consumed item-3
Produced item-5
Consumed item-4
Produced item-6
Consumed item-5
Produced item-7
Consumed item-6
Produced item-8
Consumed item-7
Produced item-9
Consumed item-8
Consumed item-9


In [64]:
import asyncio
import random
import string

# Define an asynchronous listener that constantly listens for tasks.
async def listener(task_queue: asyncio.Queue, result_queue: asyncio.Queue):
    while True:
        # Wait for a task from the task_queue
        task = await task_queue.get()
        if task is None:
            break  # Exit if a None task is received
        
        # Perform the task (simulated by sleeping for a random time)
        print(f"Listener: Performing task {task}")
        await asyncio.sleep(random.uniform(0.5, 2.0))
        
        # Simulate a result
        result = f"Result of {task}"
        
        # Put the result in the result_queue
        await result_queue.put(result)
        print(f"Listener: Finished task {task}, result {result}")

# Define an asynchronous caller that adds tasks to the task_queue and retrieves results from the result_queue.
async def caller(task_queue: asyncio.Queue, result_queue: asyncio.Queue, task_id: int):
    # Generate a task name
    task = f"task_{task_id}"
    
    # Add the task to the task_queue
    await task_queue.put(task)
    print(f"Caller: Added {task} to task_queue")
    
    # Wait for the result from the result_queue
    result = await result_queue.get()
    print(f"Caller: Retrieved {result} from result_queue")
    return result

async def main():
    # Create asyncio queues for tasks and results
    task_queue = asyncio.Queue()
    result_queue = asyncio.Queue()
    
    # Start the listener
    listener_task = asyncio.create_task(listener(task_queue, result_queue))
    
    # Start multiple callers
    callers = [caller(task_queue, result_queue, i) for i in range(5)]
    
    # Wait for all callers to complete
    results = await asyncio.gather(*callers)
    
    # Stop the listener by adding a None task
    await task_queue.put(None)
    
    # Wait for the listener to finish
    await listener_task

    print("All tasks completed")
    print("Results:", results)

# Run the main function
if __name__ == "__main__":
    asyncio.run(main())


Caller: Added task_0 to task_queue
Caller: Added task_1 to task_queue
Caller: Added task_2 to task_queue
Caller: Added task_3 to task_queue
Caller: Added task_4 to task_queue
Listener: Performing task task_0
Listener: Finished task task_0, result Result of task_0
Listener: Performing task task_1
Caller: Retrieved Result of task_0 from result_queue
Listener: Finished task task_1, result Result of task_1
Listener: Performing task task_2
Caller: Retrieved Result of task_1 from result_queue
Listener: Finished task task_2, result Result of task_2
Listener: Performing task task_3
Caller: Retrieved Result of task_2 from result_queue
Listener: Finished task task_3, result Result of task_3
Listener: Performing task task_4
Caller: Retrieved Result of task_3 from result_queue
Listener: Finished task task_4, result Result of task_4
Caller: Retrieved Result of task_4 from result_queue
All tasks completed
Results: ['Result of task_0', 'Result of task_1', 'Result of task_2', 'Result of task_3', 'Resu